In [1]:
import os
import cobra
import dnngior
os.environ['GRB_LICENSE_FILE'] = '../../content/licenses/gurobi.lic'

Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
The rest of dnngior features can be used without it.


In [ ]:
import os
import pandas as pd
import dnngior

# 设置路径
model_folder = '../../models/fixed_problem_models/propionate_models'
medium_file = '../../models/complete_media_without_oxygen.tsv'
objective_name = 'EX_cpd00141_e0'

# 结果列表
results = []

# 遍历模型文件夹
for filename in os.listdir(model_folder):
    if filename.endswith(".xml"):
        model_path = os.path.join(model_folder, filename)
        print(f"正在处理模型: {filename}")
        
        # gapfill
        gapfilled_model = dnngior.Gapfill(
            draftModel=model_path,
            medium_file=medium_file,
            objectiveName=objective_name
        )

        added_reactions = gapfilled_model.added_reactions  
        num_added = len(added_reactions)

        # 记录结果
        results.append({
            'model': filename,
            'num_added_reactions': num_added,
            'added_reaction_ids': ';'.join(added_reactions)
        })

# 保存为 CSV 文件
df = pd.DataFrame(results)
df.to_csv('../../results/prop_gapfill_summary.csv', index=False)

print(" gapfill_summary.csv 已生成。")



正在处理模型: MAG099_gapfilled_noO2.xml
Gap-filling database =  ModelSEED


No objective coefficients in model. Unclear what should be optimized


Loading medium from: ../../models/complete_media_without_oxygen.tsv
#reactions not found in NN-keys:  117 / 933
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  76605 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  38302 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19151 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9575 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4787 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2393 


Warning for adding constraints: zer

No objective coefficients in model. Unclear what should be optimized


Loading medium from: ../../models/complete_media_without_oxygen.tsv
#reactions not found in NN-keys:  110 / 893
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  76682 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  38341 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19170 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9585 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4792 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2396 


Warning for adding constraints: zer

...

Loading medium from: ../../models/complete_media_without_oxygen.tsv
#reactions not found in NN-keys:  117 / 1012
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  76478 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  38239 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19119 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9559 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4779 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2389 


Warning for adding constraints: ze

No objective coefficients in model. Unclear what should be optimized


Loading medium from: ../../models/complete_media_without_oxygen.tsv
#reactions not found in NN-keys:  61 / 1539
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  75578 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  37789 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  18894 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9447 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4723 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2361 


Warning for adding constraints: zer

No objective coefficients in model. Unclear what should be optimized


Loading medium from: ../../models/complete_media_without_oxygen.tsv
#reactions not found in NN-keys:  64 / 1601
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  75461 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  37730 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  18865 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9432 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4716 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2358 


Warning for adding constraints: zer

In [25]:
import pickle
reaction_dict_path = "../graph_analysis/reaction_dict.pkl"

# ===== 加载 reaction_dict =====
with open(reaction_dict_path, "rb") as f:
    react_dict = pickle.load(f)

In [28]:
import os
import pandas as pd
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

# 设置路径
model_dir = '../../models/fixed_problem_models/propionate_models'
propionate_exchange_id = "EX_cpd00141_e0"  
df = pd.read_csv("../../results/prop_gapfill_summary.csv")  


# 储存结果
flux_results = []

for idx, row in df.iterrows():
    model_file = row["model"]
    model_path = os.path.join(model_dir, model_file)

    if not os.path.exists(model_path):
        print(f"模型 {model_file} 不存在，跳过。")
        continue

    # 加载模型
    model = read_sbml_model(model_path)

    # 获取反应列表
    rxn_list_str = row.get("added_reaction_ids", "")
    if not isinstance(rxn_list_str, str) or rxn_list_str.strip() == "":
        print(f"模型 {model_file} 没有反应记录，跳过。")
        continue

    rxn_ids = rxn_list_str.split(";")

    # 添加缺失反应
    for rxn_id in rxn_ids:
        if rxn_id not in model.reactions and rxn_id in react_dict:
            model.add_reactions([react_dict[rxn_id].copy()])

    # 检查是否有丙酸exchange反应，没有就添加
    if propionate_exchange_id not in model.reactions:
        prop = Metabolite("cpd00141_e0", name="propionate", compartment="e")
        ex_rxn = Reaction(propionate_exchange_id)
        ex_rxn.name = "Exchange for propionate"
        ex_rxn.lower_bound = -1000
        ex_rxn.upper_bound = 1000
        ex_rxn.add_metabolites({prop: -1})
        model.add_reactions([ex_rxn])

    # 设置目标函数
    model.objective = propionate_exchange_id

    # 运行FBA
    try:
        sol = model.optimize()
        flux = sol.objective_value
    except Exception as e:
        print(f"模型 {model_file} 优化失败: {e}")
        flux = None

    # 保存结果
    flux_results.append({
        "model": model_file,
        "propionate_flux": flux
    })

# 输出
result_df = pd.DataFrame(flux_results)
result_df


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12


,model,propionate_flux
0,MAG099_gapfilled_noO2.xml,0.0
1,MAG018.sbml_gapfilled_noO2.xml,0.0
2,MAG071_gapfilled_noO2.xml,0.0
3,MAG041_gapfilled_noO2.xml,0.0
4,MAG189_gapfilled_noO2.xml,0.0
5,MAG010.sbml_gapfilled_noO2.xml,0.0
6,MAG005.sbml_gapfilled_noO2.xml,0.0
7,MAG038_gapfilled_noO2.xml,0.0
8,MAG070_gapfilled_noO2.xml,0.0
9,MAG106_gapfilled_noO2.xml,0.0
